# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
file=os.path.join('..','WeatherPy','output_data','Clean_City_data.csv')
df=pd.read_csv(file)
del df['Unnamed: 0']
df.head()


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,ipameri,-17.72,-48.16,62.19,73.0,1.0,4.97,BR,1.592193e+09
1,peniche,39.36,-9.38,56.62,90.0,0.0,1.81,PT,1.592193e+09
2,wahiawa,21.50,-158.02,80.73,61.0,75.0,9.17,US,1.592192e+09
3,punta arenas,-53.15,-70.92,37.20,93.0,100.0,11.41,CL,1.592192e+09
4,butaritari,3.07,172.79,83.12,71.0,56.0,4.63,KI,1.592193e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]
humidity=df["Humidity (%)"]

# Plot Heatmap
fig = gmaps.figure(map_type='TERRAIN',center=(0, 0), zoom_level=2,layout={
        'width': '1000px',
        'height': '700px',
        'padding': '3px',
        'border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
ideal_df=df.loc[(df["Max Temp (F)"]>=70) & (df["Max Temp (F)"]<=80) & (df["Wind Speed"]<=10) & (df["Cloudiness (%)"]==0),:]
ideal_df=ideal_df.reset_index(drop=True)
ideal_df.head(20)

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,namibe,-15.20,12.15,70.20,57.0,0.0,4.47,AO,1.592193e+09
1,san patricio,19.22,-104.70,78.82,78.0,0.0,5.46,MX,1.592193e+09
2,trairi,-3.28,-39.27,75.22,90.0,0.0,7.38,BR,1.592193e+09
3,alta floresta,-9.88,-56.09,70.09,68.0,0.0,3.65,BR,1.592193e+09
4,sar-e pul,35.83,66.17,76.26,18.0,0.0,2.68,AF,1.592193e+09
5,san blas,21.52,-105.27,78.04,74.0,0.0,1.28,MX,1.592193e+09
6,amarante do maranhao,-5.57,-46.74,70.72,97.0,0.0,3.06,BR,1.592193e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotels=[]

# Loop through dataframe to get nearest hotel
for city in range(len(ideal_df["City"])):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
              }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_response = requests.get(base_url, params=params).json()
 
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_df["Nearest Hotel"] = hotels
ideal_df.head()



,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date,Nearest Hotel
0,namibe,-15.20,12.15,70.20,57.0,0.0,4.47,AO,1.592193e+09,Hotel Chik Chik Namibe
1,san patricio,19.22,-104.70,78.82,78.0,0.0,5.46,MX,1.592193e+09,Hotel Cabo Blanco
2,trairi,-3.28,-39.27,75.22,90.0,0.0,7.38,BR,1.592193e+09,Pousada Field Lilies - Trairi
3,alta floresta,-9.88,-56.09,70.09,68.0,0.0,3.65,BR,1.592193e+09,Floresta Amazonica Hotel
4,sar-e pul,35.83,66.17,76.26,18.0,0.0,2.68,AF,1.592193e+09,Nearest hotel not found


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]


In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))